# Creating a lgistic regression to predict high sunshine production

## Import the relevant libraries

In [95]:
import pandas as pd
import numpy as np

In [216]:
df = pd.read_csv("C:/Users/charl/PythonWorkshop/Notebooks/Mldata.csv",engine='python')

In [217]:
df

,region_nord,region_centre,region_sud,Month Value,Year Value,Vitesse du vent ÃƒÂ 100m (m/s),Production ÃƒÂ©olienne (GWh),Rayonnement solaire global (W/m2),Production solaire (GWh)
0,1,0,0,2,2014,NaN,6.518965,NaN,2.931509
1,0,0,1,3,2014,NaN,220.644746,NaN,116.568735
2,0,0,1,4,2014,NaN,189.563606,NaN,125.730294
3,0,0,1,4,2014,NaN,9.671714,NaN,98.951709
4,1,0,0,5,2014,NaN,363.412141,NaN,58.899353
...,...,...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,0.0,NaN,5.82,NaN
1249,0,0,1,3,2021,0.0,NaN,6.74,NaN
1250,0,1,0,3,2021,0.0,NaN,7.15,NaN
1251,0,0,1,3,2021,0.0,NaN,6.04,NaN


In [218]:
df = df[df['Rayonnement solaire global (W/m2)'].notna()]

In [219]:
cols = df.columns.tolist()
cols

['region_nord',
 'region_centre',
 'region_sud',
 'Month Value',
 'Year Value',
 'Vitesse du vent ÃƒÂ\xa0 100m (m/s)',
 'Production ÃƒÂ©olienne (GWh)',
 'Rayonnement solaire global (W/m2)',
 'Production solaire (GWh)']

### Eolien

In [220]:
colse = cols[:7]

In [221]:
colse

['region_nord',
 'region_centre',
 'region_sud',
 'Month Value',
 'Year Value',
 'Vitesse du vent ÃƒÂ\xa0 100m (m/s)',
 'Production ÃƒÂ©olienne (GWh)']

In [222]:
dfe = df[colse]

In [223]:
dfe

,region_nord,region_centre,region_sud,Month Value,Year Value,Vitesse du vent ÃƒÂ 100m (m/s),Production ÃƒÂ©olienne (GWh)
47,0,1,0,1,2016,40.527611,262.374714
48,1,0,0,1,2016,37.960810,746.281139
49,0,1,0,2,2016,86.446078,164.631573
50,0,1,0,2,2016,74.723319,200.313391
51,0,0,1,2,2016,86.865388,125.600846
...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,0.000000,NaN
1249,0,0,1,3,2021,0.000000,NaN
1250,0,1,0,3,2021,0.000000,NaN
1251,0,0,1,3,2021,0.000000,NaN


In [224]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [225]:
X,y = make_regression(n_samples=1000, n_features=6,
                                 n_informative=3, n_targets=1,  
                                 tail_strength=0.5,  
                                 shuffle=True, coef=False, random_state=0)
notnans = dfe['Production ÃƒÂ©olienne (GWh)'].notnull()
df_notnans = df[notnans]
X_train, X_test, y_train, y_test = train_test_split(df_notnans[colse[:6]], df_notnans[colse[6:7]],
                                                    train_size=0.75,
                                                    random_state=10000)

In [226]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

regr_multirf = MultiOutputRegressor(RandomForestRegressor(max_depth=30,
                                                          random_state=0))

# Fit on the train data
regr_multirf.fit(X_train, y_train)

# Check the prediction score
score = regr_multirf.score(X_test, y_test)
print("The prediction score on the test data is {:.2f}%".format(score*100))

The prediction score on the test data is 12.62%


In [227]:
df_nans = dfe.loc[~notnans].copy()
df_nans[colse[6:7]] = regr_multirf.predict(df_nans[colse[0:6]])
df_nans

,region_nord,region_centre,region_sud,Month Value,Year Value,Vitesse du vent ÃƒÂ 100m (m/s),Production ÃƒÂ©olienne (GWh)
1092,1,0,0,1,2016,38.776437,430.271374
1093,0,0,0,1,2016,38.879595,212.409420
1094,1,0,0,2,2016,65.883750,293.116688
1095,0,0,0,2,2016,74.548664,199.927732
1096,1,0,0,3,2016,118.706316,232.478392
...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,0.000000,264.092258
1249,0,0,1,3,2021,0.000000,264.092258
1250,0,1,0,3,2021,0.000000,308.272595
1251,0,0,1,3,2021,0.000000,264.092258


### Solaire

In [228]:
colss = cols[:5] + cols[7:9]

In [229]:
colss

['region_nord',
 'region_centre',
 'region_sud',
 'Month Value',
 'Year Value',
 'Rayonnement solaire global (W/m2)',
 'Production solaire (GWh)']

In [230]:
dfs = df[colss]

In [231]:
dfs

,region_nord,region_centre,region_sud,Month Value,Year Value,Rayonnement solaire global (W/m2),Production solaire (GWh)
47,0,1,0,1,2016,7.466964,7.040643
48,1,0,0,1,2016,8.295263,5.596140
49,0,1,0,2,2016,9.242543,23.337292
50,0,1,0,2,2016,9.946336,9.640264
51,0,0,1,2,2016,6.508233,116.805294
...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,5.820000,NaN
1249,0,0,1,3,2021,6.740000,NaN
1250,0,1,0,3,2021,7.150000,NaN
1251,0,0,1,3,2021,6.040000,NaN


In [232]:
X,y = make_regression(n_samples=10000, n_features=6,
                                 n_informative=3, n_targets=1,  
                                 tail_strength=0.5,  
                                 shuffle=True, coef=False, random_state=0)
notnans = dfs['Production solaire (GWh)'].notnull()
df_notnans = df[notnans]
X_train, X_test, y_train, y_test = train_test_split(df_notnans[colss[:6]], df_notnans[colss[6:7]],
                                                    train_size=0.75,
                                                    random_state=1000)

In [233]:
regr_multirf = MultiOutputRegressor(RandomForestRegressor(max_depth=30,
                                                          random_state=0))

# Fit on the train data
regr_multirf.fit(X_train, y_train)

# Check the prediction score
score = regr_multirf.score(X_test, y_test)
print("The prediction score on the test data is {:.2f}%".format(score*100))

The prediction score on the test data is 49.60%


In [234]:
df_nans = dfs.loc[~notnans].copy()
df_nans[colss[6:7]] = regr_multirf.predict(df_nans[colss[0:6]])
df_nans

,region_nord,region_centre,region_sud,Month Value,Year Value,Rayonnement solaire global (W/m2),Production solaire (GWh)
1092,1,0,0,1,2016,6.130850,10.600893
1093,0,0,0,1,2016,7.882551,6.865031
1094,1,0,0,2,2016,6.926638,7.970764
1095,0,0,0,2,2016,8.113793,11.261300
1096,1,0,0,3,2016,5.711842,15.387454
...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,5.820000,212.833877
1249,0,0,1,3,2021,6.740000,211.701197
1250,0,1,0,3,2021,7.150000,31.504386
1251,0,0,1,3,2021,6.040000,175.097647


# Create the targets

In [284]:
targets = np.where(df_nans['Production solaire (GWh)'] > df_nans['Production solaire (GWh)'].median(), 1, 0)

In [283]:
targets.shape

(161,)

In [237]:
df_nans['Execessive Solar'] = targets

# A comment on the targets

In [238]:
targets.sum() / targets.shape[0]

0.4968944099378882

In [239]:
data_with_targets = df_nans

In [240]:
data_with_targets

,region_nord,region_centre,region_sud,Month Value,Year Value,Rayonnement solaire global (W/m2),Production solaire (GWh),Execessive Solar
1092,1,0,0,1,2016,6.130850,10.600893,0
1093,0,0,0,1,2016,7.882551,6.865031,0
1094,1,0,0,2,2016,6.926638,7.970764,0
1095,0,0,0,2,2016,8.113793,11.261300,0
1096,1,0,0,3,2016,5.711842,15.387454,0
...,...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,5.820000,212.833877,1
1249,0,0,1,3,2021,6.740000,211.701197,1
1250,0,1,0,3,2021,7.150000,31.504386,1
1251,0,0,1,3,2021,6.040000,175.097647,1


# Select the inputs for the regression

In [241]:
data_with_targets.shape

(161, 8)

In [242]:
unscaled_inputs = data_with_targets.iloc[:,:-1]
unscaled_inputs

,region_nord,region_centre,region_sud,Month Value,Year Value,Rayonnement solaire global (W/m2),Production solaire (GWh)
1092,1,0,0,1,2016,6.130850,10.600893
1093,0,0,0,1,2016,7.882551,6.865031
1094,1,0,0,2,2016,6.926638,7.970764
1095,0,0,0,2,2016,8.113793,11.261300
1096,1,0,0,3,2016,5.711842,15.387454
...,...,...,...,...,...,...,...
1248,0,0,1,3,2021,5.820000,212.833877
1249,0,0,1,3,2021,6.740000,211.701197
1250,0,1,0,3,2021,7.150000,31.504386
1251,0,0,1,3,2021,6.040000,175.097647


# Standardize the data

In [243]:
#from sklearn.preprocessing import StandardScaler

solar_scaler = StandardScaler(copy=True, with_mean=True, with_std=True)

In [244]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.vr_ = None
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled= X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled,X_scaled], axis=1)[init_col_order]

In [245]:
unscaled_inputs.columns.values

array(['region_nord', 'region_centre', 'region_sud', 'Month Value',
       'Year Value', 'Rayonnement solaire global (W/m2)',
       'Production solaire (GWh)'], dtype=object)

In [310]:
columns_to_scale = ['region_sud', 'region_nord', 'region_centre', 'Month Value',
       'Year Value', 'Rayonnement solaire global (W/m2)', 'Production solaire (GWh)']
columns_to_omit = []

In [311]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [312]:
solar_scaler = CustomScaler(columns_to_scale)

In [313]:
solar_scaler.fit(unscaled_inputs)

C:\Users\charl\anaconda3\lib\site-packages\sklearn\base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CustomScaler(columns=['region_nord', 'region_centre', 'region_sud',
                      'Month Value', 'Year Value',
                      'Rayonnement solaire global (W/m2)',
                      'Production solaire (GWh)'],
             copy=None, with_mean=None, with_std=None)

In [314]:
scaled_inputs = solar_scaler.transform(unscaled_inputs)

In [315]:
solar_scaler.fit(unscaled_inputs)

C:\Users\charl\anaconda3\lib\site-packages\sklearn\base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CustomScaler(columns=['region_nord', 'region_centre', 'region_sud',
                      'Month Value', 'Year Value',
                      'Rayonnement solaire global (W/m2)',
                      'Production solaire (GWh)'],
             copy=None, with_mean=None, with_std=None)

In [316]:
scaled_inputs = solar_scaler.transform(unscaled_inputs)

In [317]:
scaled_inputs[:161][:]

,region_nord,region_centre,region_sud,Month Value,Year Value,Rayonnement solaire global (W/m2),Production solaire (GWh)
0,1.097943,-0.283790,-0.320530,-1.204950,-1.545362,0.262034,-0.545660
1,-0.910794,-0.283790,-0.320530,-1.204950,-1.545362,1.672103,-0.652001
2,1.097943,-0.283790,-0.320530,-0.927415,-1.545362,0.902621,-0.620526
3,-0.910794,-0.283790,-0.320530,-0.927415,-1.545362,1.858247,-0.526862
4,1.097943,-0.283790,-0.320530,-0.649880,-1.545362,-0.075255,-0.409411
...,...,...,...,...,...,...,...
156,-0.910794,-0.283790,3.119829,-0.649880,1.250181,0.011809,5.210866
157,-0.910794,-0.283790,3.119829,-0.649880,1.250181,0.752383,5.178625
158,-0.910794,3.523729,-0.320530,-0.649880,1.250181,1.082421,0.049354
159,-0.910794,-0.283790,3.119829,-0.649880,1.250181,0.188903,4.136711


In [318]:
scaled_inputs[:161][:].shape

(161, 7)

# Split the data into train & test and shuffle

## Import the relevant module 

In [319]:
from sklearn.model_selection import train_test_split

## Split

In [320]:
train_test_split(scaled_inputs[:161][:], targets)

[     region_nord  region_centre  region_sud  Month Value  Year Value  \
 137    -0.910794       3.523729   -0.320530    -0.927415    1.250181   
 5      -0.910794      -0.283790   -0.320530    -0.649880   -1.545362   
 144    -0.910794      -0.283790    3.119829    -0.927415    1.250181   
 135    -0.910794      -0.283790    3.119829    -0.927415    1.250181   
 114     1.097943      -0.283790   -0.320530     1.292865    0.691072   
 ..           ...            ...         ...          ...         ...   
 104     1.097943      -0.283790   -0.320530    -0.094810    0.691072   
 33     -0.910794      -0.283790   -0.320530    -0.094810   -0.986254   
 48      1.097943      -0.283790   -0.320530    -1.204950   -0.427145   
 32      1.097943      -0.283790   -0.320530    -0.094810   -0.986254   
 47     -0.910794      -0.283790   -0.320530     1.847935   -0.986254   
 
      Rayonnement solaire global (W/m2)  Production solaire (GWh)  
 137                           2.498775               

In [321]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs[:161][:], targets, train_size = 0.8, random_state = 20)

In [322]:
print(x_train.shape, y_train.shape)

(128, 7) (128,)


In [323]:
print(x_test.shape, y_test.shape)

(33, 7) (33,)


# Logisitc regression with sklearn

In [324]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Training the model

In [325]:
reg = LogisticRegression()

In [326]:
reg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [327]:
reg.score(x_train, y_train)

0.921875

### Manually check accuracy

In [328]:
model_outputs = reg.predict(x_train)
model_outputs

array([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0])

In [329]:
y_train

array([1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0])

In [330]:
model_outputs == y_train

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True, False,
        True,  True,  True,  True, False,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True]

In [331]:
np.sum(model_outputs==y_train)

118

In [332]:
model_outputs.shape[0]

128

In [333]:
np.sum(model_outputs==y_train)/model_outputs.shape[0]

0.921875

### Finding the intercept and coefficients

In [334]:
reg.intercept_

array([0.59627108])

In [335]:
reg.coef_

array([[-1.31345511, -0.32669136,  0.0029981 , -0.33367046,  1.28124293,
        -1.98692308,  2.76522689]])

In [336]:
unscaled_inputs.columns.values

array(['region_nord', 'region_centre', 'region_sud', 'Month Value',
       'Year Value', 'Rayonnement solaire global (W/m2)',
       'Production solaire (GWh)'], dtype=object)

In [337]:
feature_name = unscaled_inputs.columns.values

In [338]:
summary_table = pd.DataFrame (columns=['feature_name'], data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table

,feature_name,Coefficient
0,region_nord,-1.313455
1,region_centre,-0.326691
2,region_sud,0.002998
3,Month Value,-0.333670
4,Year Value,1.281243
5,Rayonnement solaire global (W/m2),-1.986923
6,Production solaire (GWh),2.765227


In [339]:
summary_table.index = summary_table.index+1

In [340]:
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,feature_name,Coefficient
0,Intercept,0.596271
1,region_nord,-1.313455
2,region_centre,-0.326691
3,region_sud,0.002998
4,Month Value,-0.333670
5,Year Value,1.281243
6,Rayonnement solaire global (W/m2),-1.986923
7,Production solaire (GWh),2.765227


## Interpreting the coefficient

In [341]:
summary_table['Odd_ratio'] = np.exp(summary_table.Coefficient)

In [342]:
summary_table

,feature_name,Coefficient,Odd_ratio
0,Intercept,0.596271,1.815337
1,region_nord,-1.313455,0.268889
2,region_centre,-0.326691,0.721306
3,region_sud,0.002998,1.003003
4,Month Value,-0.333670,0.716290
5,Year Value,1.281243,3.601113
6,Rayonnement solaire global (W/m2),-1.986923,0.137117
7,Production solaire (GWh),2.765227,15.882643


In [343]:
summary_table.sort_values('Odd_ratio', ascending=False)

,feature_name,Coefficient,Odd_ratio
7,Production solaire (GWh),2.765227,15.882643
5,Year Value,1.281243,3.601113
0,Intercept,0.596271,1.815337
3,region_sud,0.002998,1.003003
2,region_centre,-0.326691,0.721306
4,Month Value,-0.333670,0.716290
1,region_nord,-1.313455,0.268889
6,Rayonnement solaire global (W/m2),-1.986923,0.137117


# Testing the model

In [344]:
reg.score(x_test, y_test)

0.7575757575757576

In [345]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[1.68709188e-02, 9.83129081e-01],
       [8.90675667e-01, 1.09324333e-01],
       [6.18210300e-01, 3.81789700e-01],
       [3.01830820e-01, 6.98169180e-01],
       [7.26718081e-01, 2.73281919e-01],
       [1.99758156e-01, 8.00241844e-01],
       [8.40589050e-01, 1.59410950e-01],
       [6.52033684e-01, 3.47966316e-01],
       [7.73414120e-01, 2.26585880e-01],
       [1.77732614e-01, 8.22267386e-01],
       [5.82542261e-01, 4.17457739e-01],
       [5.50885139e-01, 4.49114861e-01],
       [3.02687010e-01, 6.97312990e-01],
       [6.56846505e-08, 9.99999934e-01],
       [9.96357484e-01, 3.64251638e-03],
       [7.99766098e-01, 2.00233902e-01],
       [5.14309228e-06, 9.99994857e-01],
       [1.02965034e-01, 8.97034966e-01],
       [7.76574257e-05, 9.99922343e-01],
       [6.52966702e-01, 3.47033298e-01],
       [6.84086246e-01, 3.15913754e-01],
       [9.75678084e-01, 2.43219165e-02],
       [9.63731198e-01, 3.62688018e-02],
       [1.78246005e-01, 8.21753995e-01],
       [6.665626

In [346]:
predicted_proba.shape

(33, 2)

In [347]:
predicted_proba[:,1]

array([0.98312908, 0.10932433, 0.3817897 , 0.69816918, 0.27328192,
       0.80024184, 0.15941095, 0.34796632, 0.22658588, 0.82226739,
       0.41745774, 0.44911486, 0.69731299, 0.99999993, 0.00364252,
       0.2002339 , 0.99999486, 0.89703497, 0.99992234, 0.3470333 ,
       0.31591375, 0.02432192, 0.0362688 , 0.821754  , 0.93334373,
       0.974304  , 0.00274672, 0.84284659, 0.1090137 , 0.13079753,
       0.20391505, 0.27108358, 0.42617393])

# Saving the model

In [56]:
import pickle

In [58]:
with open('model','wb') as file:
    pickle.dump(reg, file)

In [59]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)